## Step 1: Training Setup

In [1]:
import torch
import torch.nn as nn
import math
import torch.utils.data as data
import numpy as np
import os
import requests
import time
import sys

from torchvision import transforms
from pycocotools.coco import COCO

from utils.data_loader import get_loader
from utils.model import EncoderCNN, DecoderRNN

In [2]:
## Select appropriate values for the Python variables below.
batch_size = 32          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'logs/training_log.txt'       # name of file with saved training loss and perplexity

In [3]:
# Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

In [4]:
# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
Obtaining caption lengths...


In [5]:
# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

In [6]:
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

In [7]:
# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

DecoderRNN(
  (embedding): Embedding(9955, 512)
  (lstm): LSTM(512, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=9955, bias=True)
)

In [8]:
# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

In [9]:
# Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

In [10]:
# Define the optimizer.
optimizer = torch.optim.ASGD(params, lr=0.01)

In [11]:
# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

## Step 2: Train your Model

In [12]:
f = open(log_file, 'w')

In [ ]:
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/12942], Loss: 9.0331, Perplexity: 8376.1209
Epoch [1/3], Step [200/12942], Loss: 8.7504, Perplexity: 6313.2946
Epoch [1/3], Step [300/12942], Loss: 8.3646, Perplexity: 4292.4280
Epoch [1/3], Step [400/12942], Loss: 7.5638, Perplexity: 1927.2382
Epoch [1/3], Step [500/12942], Loss: 7.2728, Perplexity: 1440.5976
Epoch [1/3], Step [600/12942], Loss: 7.1054, Perplexity: 1218.5400
Epoch [1/3], Step [700/12942], Loss: 6.8978, Perplexity: 990.10737
Epoch [1/3], Step [800/12942], Loss: 6.1229, Perplexity: 456.19726
Epoch [1/3], Step [900/12942], Loss: 6.0771, Perplexity: 435.76469
Epoch [1/3], Step [1000/12942], Loss: 6.1330, Perplexity: 460.8360
Epoch [1/3], Step [1100/12942], Loss: 6.2891, Perplexity: 538.6807
Epoch [1/3], Step [1200/12942], Loss: 5.3247, Perplexity: 205.3390
Epoch [1/3], Step [1300/12942], Loss: 5.5585, Perplexity: 259.4231
Epoch [1/3], Step [1400/12942], Loss: 5.5357, Perplexity: 253.5978
Epoch [1/3], Step [1500/12942], Loss: 5.0822, Perplexity: 161.